In [36]:
import os
import pandas as pd
import numpy as np

In [51]:
# load metadata table
meta = pd.read_csv('metadata_nwb_introduction_v1.csv')

In [52]:
# take a look at the top of the table
meta.head()

,Eartag,AssayType,RecordingDate,Sex,Genotype,DOB,Family,SurgeryDate,TDTGroupPath,TDTTankPath,...,ScoreFile,PreprocessedDataPath,PreprocessedFiledFF,PreprocessedFileTrimdFF,PartnerEartag,PartnerSex,PartnerGenotype,PartnerDOB,PartnerFamily,PairDate
0,V1259,introduction,20181205,M,WT,20181001,FC,20181115,M:\photometry-Long2025\tdt_files\1 Intros,NaN,...,NaN,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20181205
1,V2210,introduction,20190218,M,WT,20181223,B5,20190125,M:\photometry-Long2025\tdt_files\1 Intros,NaN,...,NaN,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190218
2,V2211,introduction,20190218,M,WT,20181223,B5,20190130,M:\photometry-Long2025\tdt_files\1 Intros,NaN,...,NaN,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190218
3,V3950,introduction,20190904,M,WT,20190706,G15,20190808,M:\photometry-Long2025\tdt_files\1 Intros,NaN,...,NaN,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,NaN,NaN,V4146,F,WT,20190726.0,NG5,20190904
4,V5473,introduction,20200212,M,WT,20191213,G21,20200123,M:\photometry-Long2025\tdt_files\1 Intros,NaN,...,NaN,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,NaN,NaN,V5552,F,WT,20191223.0,H5,20200212


In [66]:
# add a column of TDT tank directories

matchdirs = []

for i,eartag in enumerate(meta.Eartag):
    # grab the path to the TDT tanks
    tdtpath = meta.TDTGroupPath[i]
    matchdirs += [each for each in os.listdir(tdtpath) if each.lower().startswith(eartag.lower())]
    
meta.TDTTankPath = matchdirs

In [67]:
# add column of video paths

joinedpaths = []

for i,tank in enumerate(meta.TDTTankPath):
    jpath = os.path.join(meta.TDTGroupPath[i],tank)
    joinedpaths.append(jpath)
    
meta.VideoPath = joinedpaths

In [68]:
# add column of video files

# all the TDT videos are AVI

allvids = []

for i,vpath in enumerate(meta.VideoPath):
    vids = []
    vids += [each for each in os.listdir(vpath) if np.logical_and(each.endswith('avi'),not each.startswith('.'))]
    if len(vids)!=1:
        print(f'Problem with tank at {vpath}')
    else:
        allvids.append(vids[0])
        
meta.VideoFile = allvids

In [69]:
# add column of score files

allscores = []

for i,eartag in enumerate(meta.Eartag):
    allscores += [each for each in os.listdir(meta.ScorePath[i]) if each.startswith(eartag)]
    
meta.ScoreFile = allscores

In [70]:
# add column of dFF files

alldff = []

for i,eartag in enumerate(meta.Eartag):
    alldff += [each for each in os.listdir(meta.PreprocessedDataPath[i]) if np.logical_and(each.startswith(eartag),each.endswith('Intro_dFF.mat'))]
    
meta.PreprocessedFiledFF = alldff

In [71]:
# add column of trimmed dFF files

alltrim = []

for i,eartag in enumerate(meta.Eartag):
    alltrim += [each for each in os.listdir(meta.PreprocessedDataPath[i]) if np.logical_and(each.startswith(eartag),each.endswith('Trim_dFF.mat'))]
    
meta.PreprocessedFileTrimdFF = alltrim    

In [73]:
# write filled in metadata to csv

meta.to_csv('metadata_nwb_introduction_v2.csv',index=False) # USE THIS FILE FOR DOWNSTREAM STUFF